In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


# FIRST KAGGLE SUBMISSION BY TEJAS

STEPS TO BE FOLLOWED:

* DATA PREPROCESSING
* EDA
* RANDOM FOREST MODEL
* ACCURACY METRIC

In [2]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender = pd.read_csv("../input/titanic/gender_submission.csv")

In [3]:
train.describe()
#AGE has mising values

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [5]:
test.Age.fillna(test.Age.mean(), inplace=True)

In [6]:
test.Fare.fillna(test.Fare.mean(), inplace=True)
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.634534,0.896760,0.981429,55.840500
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,30.272590,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [7]:
train.Age.fillna(train.Age.mean(), inplace=True)
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Looking at the features we see:
* PassengerID: Irrelevant to final fate of the passenger.
* Survived: Feature to be predicted.
* Name, Ticket, Cabin: Might be useful, for the first I am going to neglect.

The current plan is to apply a random forest regressor on the remaining features with no feature engineering and a little EDA for fun trivia.

In [10]:
train.Pclass.unique()
#Ordinal feature as first class people would have a better chance of surviving.

array([3, 1, 2])

In [11]:
train.Sex.unique()
#Categorial (duh!)

array(['male', 'female'], dtype=object)

In [12]:
train.Age.dtype
#Age is numeric

dtype('float64')

In [13]:
train.Parch.unique()

array([0, 1, 2, 5, 3, 4, 6])

In [14]:
train.SibSp.unique()

array([1, 0, 3, 4, 2, 5, 8])

In [15]:
train.Fare.dtype

dtype('float64')

In [16]:
train.Embarked.unique()
# C = Cherbourg, Q = Queenstown, S = Southampton

array(['S', 'C', 'Q', nan], dtype=object)

In [17]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [18]:
#LABEL ENCODING OF SEX FEATURE
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train.Sex = le.fit_transform(train.Sex)
test.Sex = le.fit_transform(test.Sex)
train.Sex.unique()

array([1, 0])

In [19]:
train.Embarked = le.fit_transform(train.Embarked.astype(str))
test.Embarked = le.fit_transform(test.Embarked.astype(str))
train.Embarked.unique()

array([2, 0, 1, 3])

In [20]:
train.drop(['Name','Ticket','Cabin', 'PassengerId'], axis='columns', inplace=True)
test.drop(['Name','Ticket','Cabin'], axis='columns', inplace=True)

In [21]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [22]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,1
1,893,3,0,47.0,1,0,7.0000,2
2,894,2,1,62.0,0,0,9.6875,1
3,895,3,1,27.0,0,0,8.6625,2
4,896,3,0,22.0,1,1,12.2875,2


In [23]:
X = train.drop('Survived', axis='columns')
y = train.Survived

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

In [25]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(X_train)
mm_scaler.transform(X_test)
mm_scaler.transform(test.drop('PassengerId', axis='columns'))

array([[1.        , 1.        , 0.42644649, ..., 0.        , 0.01528158,
        0.33333333],
       [1.        , 0.        , 0.58401614, ..., 0.        , 0.01366309,
        0.66666667],
       [0.5       , 1.        , 0.77309971, ..., 0.        , 0.01890874,
        0.33333333],
       ...,
       [1.        , 1.        , 0.47686878, ..., 0.        , 0.01415106,
        0.66666667],
       [1.        , 1.        , 0.37315757, ..., 0.        , 0.01571255,
        0.66666667],
       [1.        , 1.        , 0.37315757, ..., 0.16666667, 0.0436405 ,
        0.        ]])

In [26]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

RandomForestClassifier()

In [27]:
y_pred = clf.predict(X_test)

In [28]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7966101694915254


In [29]:
fin_prediction = clf.predict(test.drop('PassengerId', axis='columns'))

In [30]:
fin_prediction.shape

(418,)

In [31]:
data = {'PassengerId': test['PassengerId'], 
        'Survived': fin_prediction} 

In [32]:
final = pd.DataFrame(data)

In [33]:
final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [34]:
final.to_csv('tejas', index = False)